In [9]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    """
    Generate a feasible bus planning schedule that minimizes the number of buses used.
    
    Parameters:
    dienstregeling (pandas.DataFrame): DataFrame containing the schedule data with columns:
        'startlocatie', 'vertrektijd', 'eindlocatie', 'buslijn'
    afstandsmatrix (pandas.DataFrame): DataFrame containing distance data with columns:
        'startlocatie', 'eindlocatie', 'min reistijd in min', 'afstand in meters', 'buslijn'
    
    Returns:
    pandas.DataFrame: DataFrame containing the feasible schedule with columns:
        'startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'activiteit', 'buslijn', 'energieverbruik', 'starttijd datum', 'eindtijd datum', 'omloop nummer'
    """
    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )
    
    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')
    
    # Initialize the bus schedules and state of charge (SOC) for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    
    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(row['vertrektijd'], format='%H:%M')
        
        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)
        
        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=row['min reistijd in min'])
        arrival_time = departure_time + travel_time
        
        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available and has sufficient SOC
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption:
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = len(bus_soc)
        
        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit',
            'buslijn': bus_line,
            'energieverbruik': energy_consumption,
            'starttijd datum': departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
            'omloop nummer': available_bus
        })
        
        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption
    
    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.
    
    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)

print(f"Consolidated plan saved to {output_file}")


Consolidated plan saved to feasible_plan_consolidated.xlsx


In [10]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    """
    Generate a feasible bus planning schedule that minimizes the number of buses used.
    
    Parameters:
    dienstregeling (pandas.DataFrame): DataFrame containing the schedule data with columns:
        'startlocatie', 'vertrektijd', 'eindlocatie', 'buslijn'
    afstandsmatrix (pandas.DataFrame): DataFrame containing distance data with columns:
        'startlocatie', 'eindlocatie', 'min reistijd in min', 'afstand in meters', 'buslijn'
    
    Returns:
    pandas.DataFrame: DataFrame containing the feasible schedule with columns:
        'startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'activiteit', 'buslijn', 'energieverbruik', 'starttijd datum', 'eindtijd datum', 'omloop nummer'
    """
    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )
    
    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')
    
    # Initialize the bus schedules and state of charge (SOC) for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    
    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(row['vertrektijd'], format='%H:%M')
        
        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)
        
        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=row['min reistijd in min'])
        arrival_time = departure_time + travel_time
        
        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available and has sufficient SOC
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption:
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = len(bus_soc)
        
        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit',
            'buslijn': bus_line,
            'energieverbruik': energy_consumption,
            'starttijd datum': departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
            'omloop nummer': available_bus
        })
        
        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption
    
    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.
    
    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)

print(f"Consolidated plan saved to {output_file}")


Consolidated plan saved to feasible_plan_consolidated.xlsx


In [ ]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    """
    Generate a feasible bus planning schedule that minimizes the number of buses used and ensures buses have sufficient charge.

    Parameters:
    dienstregeling (pandas.DataFrame): DataFrame containing the schedule data with columns:
        'startlocatie', 'vertrektijd', 'eindlocatie', 'buslijn'
    afstandsmatrix (pandas.DataFrame): DataFrame containing distance data with columns:
        'startlocatie', 'eindlocatie', 'min reistijd in min', 'max reistijd in min', 'afstand in meters', 'buslijn'

    Returns:
    pandas.DataFrame: DataFrame containing the feasible schedule with columns:
        'startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'activiteit', 'buslijn', 'energieverbruik', 'starttijd datum', 'eindtijd datum', 'omloop nummer'
    """
    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'EHVGAR')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(row['vertrektijd'], format='%H:%M')

        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)

        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=row['min reistijd in min'])
        arrival_time = departure_time + travel_time

        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available, has sufficient SOC, and is not currently charging
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption and not any(charging['starttijd'] <= departure_time.time() <= charging['eindtijd'] for charging in bus_charging[bus]):
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = bus_next_omloop
            bus_next_omloop += 1

        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit' if bus_line else 'materiaal rit',
            'buslijn': bus_line if bus_line else 'n/a',
            'energieverbruik': -energy_consumption if end_location == 'EHVGAR' else energy_consumption,
            'starttijd datum': departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
            'omloop nummer': available_bus
        })

        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption

        # Check if the bus needs to charge
        if bus_soc[available_bus] < 0.1 * 300:  # 10% of battery capacity
            # Calculate the charging time required to reach 90% SOC
            charging_time = timedelta(hours=(0.9 * 300 - bus_soc[available_bus]) / 150)  # Assuming 150 kW charging rate
            charging_start = arrival_time
            charging_end = charging_start + charging_time

            # Record the charging schedule
            bus_charging[available_bus].append({
                'startlocatie': 'EHVGAR',
                'eindlocatie': 'EHVGAR',
                'starttijd': charging_start.time(),
                'eindtijd': charging_end.time(),
                'activiteit': 'opladen',
                'buslijn': 'n/a',
                'energieverbruik': -(0.9 * 300 - bus_soc[available_bus]),
                'starttijd datum': charging_start.strftime('%Y-%m-%d %H:%M:%S'),
                'eindtijd datum': charging_end.strftime('%Y-%m-%d %H:%M:%S'),
                'omloop nummer': available_bus
            })

            # Update the bus SOC after charging
            bus_soc[available_bus] = 0.9 * 300

        # Check if the bus needs to travel to the garage for the next trip
        if bus_last_location[available_bus] != 'EHVGAR' and end_location != 'EHVGAR':
            # Schedule a material trip from the last location to the garage
            garage_trip_info = afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'EHVGAR')]

            if not garage_trip_info.empty:
                garage_trip_distance = garage_trip_info['afstand in meters'].values[0]
                garage_trip_energy = calculate_energy_consumption(garage_trip_distance)
                garage_trip_travel_time = timedelta(minutes=garage_trip_info['min reistijd in min'].values[0])
                garage_trip_arrival = arrival_time + garage_trip_travel_time

                bus_schedules.append({
                    'startlocatie': bus_last_location[available_bus],
                    'eindlocatie': 'EHVGAR',
                    'starttijd': arrival_time.time(),
                    'eindtijd': garage_trip_arrival.time(),
                    'activiteit': 'materiaal rit',
                    'buslijn': 'n/a',
                    'energieverbruik': garage_trip_energy,
                    'starttijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'eindtijd datum': garage_trip_arrival.strftime('%Y-%m-%d %H:%M:%S'),
                    'omloop nummer': available_bus
                })

                # Update the bus SOC after the material trip
                bus_soc[available_bus] -= garage_trip_energy
            else:
                # Log a message if no route is found
                print(f"No route found from {bus_last_location[available_bus]} to EHVGAR for bus {available_bus}.")

        # Update the last location of the bus
        bus_last_location[available_bus] = end_location

    # Combine the bus schedules and charging schedules
    for bus, charging_trips in bus_charging.items():
        bus_schedules.extend(charging_trips)

    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.

    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)

print(f"Consolidated plan saved to {output_file}")


In [ ]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(row['vertrektijd'], format='%H:%M')

        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)

        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=int(row['min reistijd in min']))  # Convert to int
        arrival_time = departure_time + travel_time

        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available, has sufficient SOC, and is not currently charging
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption and not any(charging['starttijd'] <= departure_time.time() <= charging['eindtijd'] for charging in bus_charging[bus]):
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = bus_next_omloop
            bus_next_omloop += 1

        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit' if bus_line else 'materiaal rit',
            'buslijn': bus_line if bus_line else 'n/a',
            'energieverbruik': -energy_consumption if end_location == 'ehvgar' else energy_consumption,
            'starttijd datum': departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
            'omloop nummer': available_bus
        })

        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption

        # Check if the bus needs to charge
        if bus_soc[available_bus] < 0.1 * 300:  # 10% of battery capacity
            # Calculate the charging time required to reach 90% SOC
            charging_time = timedelta(hours=(0.9 * 300 - bus_soc[available_bus]) / 150)  # Assuming 150 kW charging rate
            charging_start = arrival_time
            charging_end = charging_start + charging_time

            # Record the charging schedule
            bus_charging[available_bus].append({
                'startlocatie': 'ehvgar',
                'eindlocatie': 'ehvgar',
                'starttijd': charging_start.time(),
                'eindtijd': charging_end.time(),
                'activiteit': 'opladen',
                'buslijn': 'n/a',
                'energieverbruik': -(0.9 * 300 - bus_soc[available_bus]),
                'starttijd datum': charging_start.strftime('%Y-%m-%d %H:%M:%S'),
                'eindtijd datum': charging_end.strftime('%Y-%m-%d %H:%M:%S'),
                'omloop nummer': available_bus
            })

            # Update the bus SOC after charging
            bus_soc[available_bus] = 0.9 * 300

        # Check if the bus needs to travel to the garage for the next trip
        if bus_last_location[available_bus] != 'ehvgar' and end_location != 'ehvgar':
            # Schedule a material trip from the last location to the garage
            garage_trip_distance = afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'ehvgar')]['afstand in meters'].values[0]
            garage_trip_energy = calculate_energy_consumption(garage_trip_distance)
            garage_trip_travel_time = timedelta(minutes=int(afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'ehvgar')]['min reistijd in min'].values[0]))
            garage_trip_arrival = arrival_time + garage_trip_travel_time

            bus_schedules.append({
                'startlocatie': bus_last_location[available_bus],
                'eindlocatie': 'ehvgar',
                'starttijd': arrival_time.time(),
                'eindtijd': garage_trip_arrival.time(),
                'activiteit': 'materiaal rit',
                'buslijn': 'n/a',
                'energieverbruik': garage_trip_energy,
                'starttijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'eindtijd datum': garage_trip_arrival.strftime('%Y-%m-%d %H:%M:%S'),
                'omloop nummer': available_bus
            })

            # Update the bus SOC after the material trip
            bus_soc[available_bus] -= garage_trip_energy

        # Update the last location of the bus
        bus_last_location[available_bus] = end_location

    # Combine the bus schedules and charging schedules
    for bus, charging_trips in bus_charging.items():
        bus_schedules.extend(charging_trips)

    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.

    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)


In [11]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    # Define the date for all trips in the schedule
    schedule_date = "2024-08-29"

    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(f"{schedule_date} {row['vertrektijd']}")

        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)

        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=int(row['min reistijd in min']))  # Convert to int
        arrival_time = departure_time + travel_time

        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available, has sufficient SOC, and is not currently charging
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption and not any(charging['starttijd'] <= departure_time.time() <= charging['eindtijd'] for charging in bus_charging[bus]):
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = bus_next_omloop
            bus_next_omloop += 1

        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit' if bus_line else 'materiaal rit',
            'buslijn': bus_line if bus_line else 'n/a',
            'energieverbruik': -energy_consumption if end_location == 'ehvgar' else energy_consumption,
            'starttijd datum': departure_time.strftime('%d/%m/%Y %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%d/%m/%Y %H:%M:%S'),
            'omloop nummer': available_bus
        })

        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption

        # Check if the bus needs to charge
        if bus_soc[available_bus] < 0.1 * 300:  # 10% of battery capacity
            # Calculate the charging time required to reach 90% SOC
            charging_time = timedelta(hours=(0.9 * 300 - bus_soc[available_bus]) / 150)  # Assuming 150 kW charging rate
            charging_start = arrival_time
            charging_end = charging_start + charging_time

            # Record the charging schedule
            bus_charging[available_bus].append({
                'startlocatie': 'ehvgar',
                'eindlocatie': 'ehvgar',
                'starttijd': charging_start.time(),
                'eindtijd': charging_end.time(),
                'activiteit': 'opladen',
                'buslijn': 'n/a',
                'energieverbruik': -(0.9 * 300 - bus_soc[available_bus]),
                'starttijd datum': charging_start.strftime('%d/%m/%Y %H:%M:%S'),
                'eindtijd datum': charging_end.strftime('%d/%m/%Y %H:%M:%S'),
                'omloop nummer': available_bus
            })

            # Update the bus SOC after charging
            bus_soc[available_bus] = 0.9 * 300

        # Check if the bus needs to travel to the garage for the next trip
        if bus_last_location[available_bus] != 'ehvgar' and end_location != 'ehvgar':
            # Schedule a material trip from the last location to the garage
            garage_trip_distance = afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'ehvgar')]['afstand in meters'].values[0]
            garage_trip_energy = calculate_energy_consumption(garage_trip_distance)
            garage_trip_travel_time = timedelta(minutes=int(afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'ehvgar')]['min reistijd in min'].values[0]))
            garage_trip_arrival = arrival_time + garage_trip_travel_time

            bus_schedules.append({
                'startlocatie': bus_last_location[available_bus],
                'eindlocatie': 'ehvgar',
                'starttijd': arrival_time.time(),
                'eindtijd': garage_trip_arrival.time(),
                'activiteit': 'materiaal rit',
                'buslijn': 'n/a',
                'energieverbruik': garage_trip_energy,
                'starttijd datum': arrival_time.strftime('%d/%m/%Y %H:%M:%S'),
                'eindtijd datum': garage_trip_arrival.strftime('%d/%m/%Y %H:%M:%S'),
                'omloop nummer': available_bus
            })

            # Update the bus SOC after the material trip
            bus_soc[available_bus] -= garage_trip_energy

        # Update the last location of the bus
        bus_last_location[available_bus] = end_location

    # Combine the bus schedules and charging schedules
    for bus, charging_trips in bus_charging.items():
        bus_schedules.extend(charging_trips)

    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.

    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)


FileNotFoundError: [Errno 2] No such file or directory: 'input_file.xlsx'